# Intro to Crypto math practical

## Purpose

Cryptography is at the heart of everything we do. The guaranteed security of cryptography is the reason we can remote into machines, make payments online, authenticate users, and more. **By the end of this practical** you should be able to:

* explain the fundamentals of modular arithmetic
* explain the mathematical principle by which RSA works
* explain how to use RSA

## Structure

This is an interactive practical. You're going to write functions in Python that accomplish key tasks. 2LT Chonofsky and CPT Faivre are available as demonstrators. You are also free to use online resources to help with the mechanics of writing code.

## RSA key generation
### Some math

RSA is the backbone of web encryption and a particularly simple example of a *public-key* encryption algorithm.

The RSA algorithm relies on the idea that you can choose two numbers ($P$ and $Q$) in some mod base which make this equation always true for any $a$.

$$
a^{P^{Q}} = a
$$

Once again: we're picking two special numbers. Then we raise $a$ to the first number and the second number and get $a$ back. This happens using modular arithmetic, i.e., on a (very large) clock face.



In [ ]:
# write a function that raises a to the power P mod N
def mod_exp(a, P, N):
  return None

mod_exp(2,20,13)

mod_exp(2,20,13):
  Value: 9
  md5: a493c3ba33a513f11fc611288e8ab649
  Correct!



OK, great! Now we need to verify some facts about modular arithmetic. Pick a mod base and a number that is larger than that base - for example, if you picked base 7, you could also pick 13. $13 = 6\ \textrm{mod}\ 7$. Then check that no matter what power $x$ you choose, $6^x\ \textrm{mod}\ 7 = 13^x\ \textrm{mod}\ 7$, so you can 'do the mod' before exponentiating.

In [ ]:
# try with some other numbers!

print(mod_exp(6,  5, 7))
print(mod_exp(13, 5, 7))

6
6


Now try computing $96^{1,000,000,000}\ \textrm{mod}\ 5$. It probably will appear not to work - it is getting stuck because the numbers involved are too large. (You can interrupt the execution by pressing `Ctrl+M I`, or selecting Interrupt in the Runtime menu.)

Thinking about the last question, how can you modify your `mod_exp` to make it work? Do so. We will need to modify `mod_exp` further later, but this is the computational core of RSA.

In [ ]:
print(mod_exp(96,1000000000,5))

1


### Generating keys

Here's the rough scheme by which we will be generating RSA keys.

1. Pick two random prime numbers (let's say: 3 and 7). Their product is $n$ (21)
2. Find a magic number $\phi(n)$ that is smaller than n. There's some complexity here - a whole lot of extra math to do - but we can avoid all that math because of the choice we made for $n$. For our special case, $\phi(n) = n - (p + q - 1)$. For example: $21 - (3+7-1)=12$.
3. Find a smaller number $e$ that has no common divisors with $\phi(n)$. $e$ along with $n$ is your public key. For example, 5 will do.
4. Find the number $d$ such that $d \times e = 1\ \textrm{mod}\ \phi(n)$. $d$ is your private key. $5\times 5 = 25 = 1\ \textrm{mod}\ 12$. 5 is the private key.


First - write a function `is_prime` that returns `True` if a number is prime and `False` otherwise. Do this by checking that the number is 0 mod every number (or every odd number) up to the square root of the number. Why don't we need to check any numbers above the square root?

In [ ]:
import math
# math.sqrt is the square root function

def is_prime(n):
  if n % 2 == 0:
    pass
  for i in range(3, math.floor(math.sqrt(n))+2, 2):
    pass
print(is_prime(18757), is_prime(18767)) # should be True, False

True False


Now we're going to use `is_prime` to randomly select two prime numbers, which we will call p and q.

In [ ]:
import random
random.seed(100)

def get_p_q():
  while True:
    p = random.randint(100000, 100000000)
    if is_prime(p):
      break
  while True:
    pass
  print(f"p = {p}")
  print(f"q = {q}")
  return p,q

p,q = get_p_q()

p = 53185283
q = 40351841


### Extended Euler's algorithm

#### Overview

Most of the next section is a complicated algorithm for transforming $p$ and $q$ into other numbers that will work with the modular exponentiation math we discussed earlier. To refresh your memory, we're looking for two numbers $P$ and $Q$, along with a $\textrm{mod}\ N$. This will allow us to take any number $a$ and raise it to $P$ and then $Q$ and get $a$ back.

#### The details

This is an algorithm for finding the greatest common divisor (gcd) of two numbers, along with the numbers $x$ and $y$ that satisfy the equation $ax + by = \textrm{gcd}(a,b)$.

We're going to use it to find the value of something called Carmichael's totient function. Then, using a related algorithm,

In [ ]:
def extended_euclid(a,b):
  if a > b: partial_diffs = [a,b]
  else: partial_diffs = [b,a]
  quotients = []
  s = [1,0]
  t = [0,1]

  while True:
    remainder = partial_diffs[-2] % partial_diffs[-1]
    quotient = partial_diffs[-2] // partial_diffs[-1]
    quotients.append(quotient)
    partial_diffs.append(remainder)
    s.append(s[-2] - quotient * s[-1])
    t.append(t[-2] - quotient * t[-1])
    if remainder == 0:
      break
  return partial_diffs, quotients, s, t

def gcd(a,b):
  diffs, _, _, _ = extended_euclid(a,b)
  return diffs[-2]

def bezout_coefficients(a,b):
  _, _, s, t = extended_euclid(a,b)
  return s[-2], t[-2]

def phi(a,b):
  return a*b - (a + b - 1)

Write a loop that selects a random number $e$ between 1 and `phi(p,q)`. Check that your number has $\textrm{gcd}(\phi(pq),e) = 1$ and pick another number if that is not true.

In [ ]:
random.seed(999)
while True:
  break
print(f"\nTogether, these two numbers form the public key:\n\ne = {e}\npq = {p*q}")


Together, these two numbers form the public key:

e = 1278542496901893
pq = 2146124083156003


Step 4 requires us to find the number $d$ that when multiplied by $e$ gives us 1 (mod $pq$). In other words, $de$ is one more than a multiple of $\phi(pq)$.
$$
de = 1 + (\textrm{a multiple of}\ \phi(pq))
$$

Let's call that multiple $\alpha$:
$$
de = 1 + \alpha(\phi(pq))
$$
Subtract $\alpha(pq)$ from both sides:
$$
de + (-\alpha)(\phi(pq)) = 1
$$

We know $e$ and $\phi(pq)$. We are trying to find $d$ and $-\alpha$, and, in fact, we don't care about $-\alpha$. There is an algorithm for doing this: the Extended Euclidean Algorithm, which gives the so-called Bézout coefficients $d$ and $-\alpha$.

Use the function `bezout_coefficients` to find $d$ and $-\alpha$:

In [ ]:
alpha, d = bezout_coefficients(e, phi(p,q))
d %= phi(p,q) # in case d is negative, make it positive
print(f"{d} is the private key")
print(f"{e} and {p*q} is the public key")

1230229340088397 is the private key
1278542496901893 and 2146124083156003 is the public key


Did we solve it? 1 should be 1


### To summarize - test it!
We did a bunch of math and got two numbers: $d$ and $e$, along with a base $pq$. For any number $a$, it should be true that
$$
\left ( a^{d} \right )^{e}= a\ \textrm{mod}\ pq
$$

**There's one problem.** Your d and e and pq will be way too large to do this computation easily. We need to turbocharge `mod_exp` using a clever algorithm before we can be done.

#### The algorithm

Let's say we want to raise 3 to the power 25. That's $3\times 3 \times 3\ \dots \times\ 3$ where the "times three" is repeated 25 times.

Instead, let's break $3^{25}$ into binary powers:

$$
3^{25} = 3^{16} \times 3^9 = 3^{16} \times 3^8 \times 3^1
$$

and we can break this down further:

$$
3^{16} \times 3^8 \times 3^1 = 3^{2^{2^{2^{2}}}} \times 3^{2^{2^2}} \times 3
$$

Since all of these numbers have to be modulo something, we're going to take the mods *as soon as possible* to keep the numbers *as small as possible*.

In [13]:

import math
def speedy_mod_exp(base, power, modulus):
    # get base**(2**max_base_two)
    # get the remaining powers still to multiply in
    # calculate max base two for the remaining powers
    # loop or (better!) recurse until max base two is 0

    pass




In [11]:
# check it!
import sys
# should happen fast
print(speedy_mod_exp(201,2000009,51200000))
sys.stdout.flush()
# should happen slow
print(       mod_exp(201,2000009,51200000))

1332904442188598882914303814092775476447514001065760753441801


Now we'll go back to our private key $d$ and public key $e$ and $pq$.

In [ ]:
# use speedy_mod_exp to raise a number of your choosing to the power d
# then raise the result to e, both mod p*q

m = (speedy_mod_exp(12,e,p*q))
print(m)
print(speedy_mod_exp(m, d, p*q))

1717481739389891
12


## Using keys!

### Level 0 - encryption and decryption

Pick a number and encrypt it with a secret key (exponentiate it to the power $d$, mod $pq$). Then share it with someone near you, along with your public key $e$ and $pq$ (the product, remember, not the individual numbers). Check that they can figure out what your first number was. Be sure to remove the `random.seed(x)` calls before you generate your number!


In [ ]:
# you can use this code block to do the computation

### Extension 1 - block cypher

Expressed as a continuous stream of bits, real-world messages would be very, very big numbers. When the size of your message exceeds $pq$, your mod base, the encryption scheme breaks because you can never recover the message from the modular arithmetic. Insted, we break the message into shorter sections which we will encrypt individually.

In [ ]:
# these are the preliminary parts - string processing - fill in the first one!

def get_blocks(text):
  # write a function that breaks a string into a list of five-character blocks
  pass

print(get_blocks('hello world')) # make sure this works: ['hello', ' worl', 'd']

def get_integer(text):
  # this is a function that takes a string and represents it as the integer that
  # is the concatenation of the two-digit hex values that represent its chars,
  # as a decimal number

  # for example  'abc' -> 0x61,0x62,0x63 -> 616263 -> 6382179
  return int( ''.join(
      [hex(ord(i)).split('x')[1] for i in text]
      ) , 16)

def get_chars(number):
  # this is a function that inverts the previous function:
  # takes a decimal number, converts it to hex, splits the hex into two-digit
  # numbers, and converts back to ascii

  chars = []
  hex_long = str(int(number, 16)).zfill(10) # fill to length ten in case leading zeros
  for x in range(0, len(hex_long), 2):
    hex_short = hex_long[x:x+2]
    chars.append(chr(int(hex_short, 16)))
  return ''.join(chars)

print(get_integer('abc'),get_chars(get_integer('abc')))

def get_integer_blocks(text):
  # use the above functions to break a longer text into integers
  blocks = get_blocks(text)
  return [get_integer(block) for block in blocks]

def get_long_string(integer_blocks):
  # inverts the above
  return ''.join([get_chars(block) for block in integer_blocks])


6382179


### Level 2 - a long message
* Encrypt the section of the Declaration of Independence (below) by splitting it into blocks of five characters at a time. Show that you can decrypt it.



### Level 3 - digital signatures
* How can you use this system to prove that you wrote a message? Use a digital signature process based in RSA of your own invention to sign the Declaration of Independence excerpt. (You may use `hashlib` to provide a hash of the text.)

In [ ]:
# enc/dec
N = 5
msg_full = ("""
He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:

For Quartering large bodies of armed troops among us:

For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:

For cutting off our Trade with all parts of the world:

For imposing Taxes on us without our Consent:

For depriving us in many cases, of the benefits of Trial by Jury:

For transporting us beyond Seas to be tried for pretended offences:""")

if len(msg_full) % N != 0:
  msg_full += ' ' * (N - len(msg_full) % N)

for x in range(0, len(msg_full), N):
  msg = msg_full[x:x+N]

  msg = '1' + ''.join([str(ord(x)).zfill(3) for x in msg])
  print(msg)

  # 950873696872261 is the private key
  # 483800656279913 and 2371655501883223 is the public key
  # encrypt

  cy = speedy_mod_exp(int(msg), 483800656279913, 2371655501883223)
  print(cy)
  decy = speedy_mod_exp(cy, 950873696872261, 2371655501883223)
  print(decy)

  recovered = str(decy)[1:]
  txt = []
  for i in range(0, len(recovered), 3):
    txt.append(chr(int(recovered[i:i+3])))
  print(''.join(txt))
  print()


1070111117114115
2128647101454650
1070111117114115
Fours

1099111114101032
919954784957861
1099111114101032
core 

1097110100032115
245075216097472
1097110100032115
and s

1101118101110032
1532281752214902
1101118101110032
even 

1121101097114115
1138277667260808
1121101097114115
years

1032097103111044
1313815995228284
1032097103111044
 ago,

1032111117114032
85501977763733
1032111117114032
 our 

1102111114101102
1998069338306761
1102111114101102
foref

1097116104101114
630916504091808
1097116104101114
ather

1115032098114111
225172777305290
1115032098114111
s bro

1117103104116032
1727768705589857
1117103104116032
ught 

1102111114116104
1844507516942075
1102111114116104
forth

1032111110032116
1717252926023558
1032111110032116
 on t

1104105115032032
705929907938126
1104105115032032
his  

